In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai.tools import tool
from enum import Enum
llm = LLM(
    model="openai/gpt-4o", # call model by provider/model_name
    stop=["END"],
    seed=42
)


class AgentPrompts(Enum):
    fundamentalAgentBackStory = """As a fundamental financial equity
analyst your primary responsibility is to analyze the most
recent 10K report provided for a company. You have access to a
powerful tool that can help you extract relevant information
from the 10K. Your analysis should be based solely on the
information that you retrieve using this tool. You can interact
with this tool using natural language queries. The tool will
understand your requests and return relevant text snippets
and data points from the 10K document. Keep checking if you
have answered the users’ question to avoid looping"""
 
    fundamentalAgentGoal = """To come up with position for a stock based on fundamental analysis. I will use the tools at your disposal to help formulate a final position"""

    valuationAgentBackStory = """As a valuation equity analyst, your primary responsibility is to analyze the valuation trends of a
given asset or portfolio over an extended time horizon. To complete the task, you must analyze the historical valuation data
of the asset or portfolio provided, identify trends and patterns in valuation metrics over time, and interpret the implications
of these trends for investors or stakeholders"""

    valuationAgentGoal = """To come up with position for a stock based on returns and volatility analysis. I will use the tools at your disposal to help formulate a final positiom"""

    sentimentAgentBackStory = """As a sentiment equity analyst your primary responsibility is to analyze the financial news, analyst
ratings and disclosures related to the underlying security;
and analyze its implication and sentiment for investors or
stakeholders"""
    sentimentAgentGoal = """ To come up with position for a stock based on sentiment analysis. I will use the tools at your disposal to help formulate a final position"""


In [ ]:
@tool
def financialReportRagTool(*args, **kwargs) -> str:
    '''
        Tool
    to get the financial report for a company
'''
    return f"Financial report is looking good"

@tool
def getAnnualisedReturnTool(*args, **kwargs) -> str:
    '''
        Get the annualised return for a company
    '''
    return 0.1

@tool
def getAnnualisedVolatilityTool(*args, **kwargs) -> str:
    '''
        Get the annualised volatility for a company
    '''
    return 0.2

@tool 
def getVolumeTool(*args, **kwargs) -> str:
    '''
        Get the volume for a company
    '''
    return 1000000

@tool 
def getNewsBodyTool(*args, **kwargs) -> str:
    '''
        Get the news body for a company
    '''
    return "we are happy with the company"



In [ ]:
from crewai import Agent, Task, Crew, Process

# Define specialist agents
valuationAgent = Agent(
    role="valuationAgent",
    goal=AgentPrompts.valuationAgentGoal.value,
    backstory=AgentPrompts.valuationAgentBackStory.value,
    tools=[getAnnualisedVolatilityTool, getAnnualisedReturnTool],
    verbose=True,
    memory=True,
    llm=llm
)

fundamentalAgent = Agent(
    role="fundamentalAgent",
    goal=AgentPrompts.fundamentalAgentGoal.value,
    backstory=AgentPrompts.fundamentalAgentBackStory.value,
    tools=[financialReportRagTool],
     verbose=True,
    memory=True,
    llm=llm
)

sentimentAgent = Agent(
    role="sentimentAgent",
    goal=AgentPrompts.sentimentAgentGoal.value,
    backstory=AgentPrompts.sentimentAgentBackStory.value,
    tools=[getNewsBodyTool],
    verbose=True,
    memory=True,
    llm=llm
)

moderator = Agent(
    role="Investment Moderator",
    goal="Keep the investment debate structured, ask for responses, and finally give a concluding investment decision.",
    backstory="You are a neutral and fair investment debate moderator with deep market experience. "
              "You ensure that each analyst has the chance to present their case and that the debate remains productive. Each analyst should provide with thier opinion on what the strategby should be and all of them must come toa final consensus that they agree on. ",
    verbose=True,
    memory=True, 
    llm=llm
)

In [ ]:

investment_debate_task = Task(
    description=(
        "Start a structured debate between the three investment analysts. "
        "Each analyst should come up with their strategy and then defend their methodology, challenge others, and respond to critiques. "
        "Ensure each agent has at least 2 turns to speak. "
        "They should be able to come to a consensus on what the final strategy should be. "
        "Topic of the debate is: 'What is the best approach to determine if we should BUY, SELL, or HOLD this stock?'"
    ),
    expected_output=(
        "A transcript of the investment debate with contributions from all 3 analysts."
    ),
    agent=moderator,  # Moderator orchestrates the discussion
)

investment_conclusion_task = Task(
    description=(
        "After the debate, analyze the discussion carefully and provide a final investment decision. "
        "The conclusion should summarize key arguments and declare the final recommendation: BUY, SELL, or HOLD "
        "with proper reasoning based on the strongest arguments presented."
    ),
    expected_output=(
        "A clear and reasoned final investment decision (BUY/SELL/HOLD) summarizing the debate "
        "and identifying which analytical approach provided the most compelling case."
    ),
    agent=moderator,
)


In [ ]:

# ------------------------
# Create Investment Crew
# ------------------------

crew = Crew(
    agents=[valuationAgent, fundamentalAgent, sentimentAgent, moderator],
    tasks=[investment_debate_task, investment_conclusion_task],
    process=Process.sequential,
)

# ------------------------
# Run Investment Debate
# ------------------------

result = crew.kickoff(inputs={})
print("Final Investment Strategy Decision:\n", result)

In [ ]:
result.raw